In [1]:
import numpy as np
from matplotlib import pyplot as plt
from skimage import io
from os.path import expanduser
HOME = expanduser("~")
import os, subprocess, sys
#%matplotlib inline

In [2]:
REPO_DIR = '/home/eddyod/programming/pipeline_utility'
sys.path.append(REPO_DIR)
from utilities.sqlcontroller import SqlController
from utilities.file_location import FileLocationManager
from utilities.metadata import ELASTIX_BIN

Connecting dklab@localhost:3306


In [3]:
stack = 'DK39'
sqlController = SqlController()
fileLocationManager = FileLocationManager(stack)
image_name_list = sqlController.get_image_list(stack, 'destination')

In [4]:
stop = len(image_name_list) - 1
generated_elastix = os.path.join(fileLocationManager.elastix_dir, 'result.0.tif')
generated_transform = os.path.join(fileLocationManager.elastix_dir, 'TransformParameters.0.txt')
params_fp = os.path.join(REPO_DIR, 'preprocess', 'parameters', 'Parameters_Rigid_MutualInfo_noNumberOfSpatialSamples_4000Iters.txt')

In [5]:
tmp_image_list = image_name_list
print(len(tmp_image_list))
tmp_image_list.insert(0, image_name_list[0])
print(len(tmp_image_list))

24
25


In [6]:
cmp_images = []
for i, file in enumerate(tmp_image_list):
    #if os.path.exists(os.path.join(fileLocationManager.elastix_dir, image_name_list[i])):
    #    continue
    moving_image = os.path.join(fileLocationManager.thumbnail_prep, tmp_image_list[i])
    if i < stop:
        fixed_image = os.path.join(fileLocationManager.thumbnail_prep, tmp_image_list[i+1])
    else:
        fixed_image = os.path.join(fileLocationManager.thumbnail_prep, tmp_image_list[-1])
        
    command = [ELASTIX_BIN, '-f', fixed_image, '-m', moving_image, '-p', params_fp, '-out', fileLocationManager.elastix_dir]
    #command = ['touch', generated_file]
    ret = subprocess.run(command)
    #command = ['touch', os.path.join(fileLocationManager.elastix_dir, generated_transform)]
    #ret = subprocess.run(command)
    transfilename = os.path.splitext(tmp_image_list[i])[0] + '.txt'
    destination_elastix = os.path.join(fileLocationManager.elastix_dir, tmp_image_list[i])
    destination_transform = os.path.join(fileLocationManager.elastix_dir, transfilename)
    os.rename(generated_elastix, destination_elastix)
    os.rename(generated_transform, destination_transform)
    cmp_images.append(tmp_image_list[i])

    #os.rename(os.path.join(fileLocationManager.elastix_dir, 'TransformParameters.0.txt'),
    #          os.path.join(fileLocationManager.elastix_dir, transfilename + '.txt' ))
    #print('command returned ', ret.returncode)


FileNotFoundError: [Errno 2] No such file or directory: '/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/DK39/preps/elastix/result.0.tif' -> '/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/DK39/preps/elastix/DK39_ID_0006_slide058_S3_C3.tif'

In [ ]:
len(cmp_images)
len(image_name_list)
[m for m in image_name_list if m not in cmp_images]